In [36]:
#implementation on iris dataset
import numpy as np
from neural_network import NeuralNetwork
from dataset import load_and_preprocess_data
from sklearn.metrics import accuracy_score

#load dataset
X_train, X_test, y_train, y_test = load_and_preprocess_data('iris.csv')

#one-hot encode the labels
y_train_onehot = np.eye(len(np.unique(y_train)))[y_train]
y_test_onehot = np.eye(len(np.unique(y_test)))[y_test]

#initialize and train the neural network
input_size = X_train.shape[1]
hidden_size = 4
output_size = y_train_onehot.shape[1]
learning_rate = 0.01

nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
nn.train(X_train, y_train_onehot, epochs=1000)

#predict on test set
y_pred = nn.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

#accuracy evaluation
accuracy = accuracy_score(y_test, y_pred_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Test accuracy: 97.78%
